In [11]:
import re
from janome.tokenizer import Tokenizer


def text_formatting(text):
    # ヘッダ，フッタ部分の除去
    text = re.split('\-{5,}', text)[2]
    text = re.split('底本：', text)[0]
    # |の除去
    text = text.replace('|', '')
    # るびの削除
    text = re.sub('《.+?》', '', text)
    # 入力注の削除
    text = re.sub('［＃.+?］', '', text)
    # 空行の削除
    text = re.sub('\n\n', '\n', text)
    text = re.sub('\r', '', text)
    return text


def extract_words(text):
    t = Tokenizer()
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens if token.part_of_speech.split(',')[0] in ['名詞', '動詞','形容詞']]

with open('./sanshiro.txt', 'r', encoding='sjis') as f:
    text = f.read()
    
text = text_formatting(text)
sentences = text.split('。')
word_list = [extract_words(sentence) for sentence in sentences]

print(word_list[0])

['一', 'する', '目', 'さめる', '女', '隣', 'じいさん', '話', '始める', 'いる']


In [14]:
from gensim.models import word2vec
# https://katsu-tech.hatenablog.com/entry/2019/01/02/201150
# gensimのバージョンを下げるとインスト―ルが成功した．

model = word2vec.Word2Vec(word_list, size=100, min_count=5, window=5, iter=100)

for item, value in model.wv.most_similar('世間'):
    print(item, value)


c:\users\yuuki\appdata\local\programs\python\python39\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\yuuki\appdata\local\programs\python\python39\lib\site-packages\gensim\models\base_any2vec.py:617: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


外国 0.5646041631698608
自己 0.5476845502853394
社会 0.5171869397163391
聞こえる 0.5012503266334534
喜ぶ 0.48332059383392334
ヘーゲル 0.48260146379470825
村 0.4812805950641632
喝采 0.48084795475006104
進む 0.4771573543548584
文学 0.474667489528656


In [15]:
model.save('sanshiro.model')

In [16]:
# https://github.com/Kyubyong/wordvectors
# https://drive.google.com/file/d/0B0ZXk88koS2KMzRjbnE4ZHJmcWM/view

import gensim


model = gensim.models.Word2Vec.load('ja.bin')
print(model)

Word2Vec(vocab=50108, size=300, alpha=0.025)


In [18]:
for item, value in model.wv.most_similar('世間'):
    print(item, value)

本心 0.5424478650093079
一般大衆 0.5371387004852295
一般社会 0.5218241214752197
マスコミ 0.5033861398696899
一般人 0.49983924627304077
迷信 0.4850359857082367
言動 0.4823036193847656
心底 0.457682341337204
偏見 0.45686981081962585
嫉妬 0.4561861753463745


In [19]:
model.wv.most_similar(positive=['東京', 'フランス'], negative=['日本'])

[('パリ', 0.5596295595169067),
 ('アムステルダム', 0.5044834017753601),
 ('ブリュッセル', 0.5014014840126038),
 ('ウィーン', 0.49867892265319824),
 ('ルーアン', 0.49242958426475525),
 ('クラクフ', 0.48978930711746216),
 ('ストラスブール', 0.48793643712997437),
 ('ベルギー', 0.48785877227783203),
 ('ナポリ', 0.48667028546333313),
 ('サンクトペテルブルク', 0.48542696237564087)]

In [20]:
model.wv.most_similar(positive=['妻', '男'], negative=['女'])

[('夫', 0.6135661005973816),
 ('雄', 0.5344775319099426),
 ('彦', 0.4987543821334839),
 ('娘', 0.479888379573822),
 ('郎', 0.4725583791732788),
 ('長男', 0.46775707602500916),
 ('らがいる', 0.4614781141281128),
 ('次男', 0.4535171389579773),
 ('三郎', 0.44910797476768494),
 ('後妻', 0.4475233554840088)]